In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");

# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# View Gen AI Agent Evaluation Run Results

This notebook demonstrates how to use the Vertex AIGen AI Eval SDK to retrieve an Evaluation Run and view the results.

* Retrieve existing Evaluation Run
* Display results

<table align="left" border="0">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/vladkol/agent-evaluation-lab/blob/main/evaluator/show_evaluation_run.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
</table>


## Install Dependencies

In [ ]:
%pip install google-cloud-aiplatform[evaluation]==1.134.* -q

## Configuration

In [ ]:
# @title Configuration
# fmt: off
GOOGLE_CLOUD_PROJECT = "" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
GOOGLE_CLOUD_REGION = "" # @param {type: "string", placeholder: "[your-region]", isTemplate: true}
EVAL_RUN_ID = "" # @param {type: "string", placeholder: "[evaluation run id]", isTemplate: true}
# fmt: on

## Authentication for Google Colab

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Retrieve Evaluation Run and Display Results

In [ ]:
from google.genai import types as genai_types
from vertexai import Client

# Initialize SDK
client = Client(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    http_options=genai_types.HttpOptions(api_version="v1beta1"),
)

evaluation_run = client.evals.get_evaluation_run(
    name=EVAL_RUN_ID,
    include_evaluation_items=True
)
evaluation_run.show()